In [8]:
import sys
sys.path.append('../bfrt_grpc')

from bfrt_grpc_client import Bfrt_GRPC_Client
import bfrt_grpc.client as gc

In [2]:
import time
import random

In [3]:
controller = Bfrt_GRPC_Client()

Subscribe attempt #1
Subscribe response received 0
Received tf_honeypot on GetForwarding on client 0, device 0
Binding with p4_name tf_honeypot
Binding with p4_name tf_honeypot successful!!


In [5]:
controller.clear_service_table()
entry_size = 30000

test_key_list = []
for i in range(entry_size):
    test_key_list.append([783662000+i, 10129, 6])
start_time = time.time()
controller.entry_batch_add(test_key_list)
print("add --- %s seconds ---" % (time.time() - start_time))
print(controller.get_table_usage())

add --- 5.862689971923828 seconds ---
30000


In [ ]:
def service_entry_get(self, entry_key_list):
    key_list = []
    expired_entries = []
    for index, key in enumerate(entry_key_list):
        key_list.append(self.service_table.make_key([gc.KeyTuple("meta.internal_ip", key[0]),
                                                    gc.KeyTuple("meta.internal_port", key[1]),
                                                    gc.KeyTuple("meta.ip_protocol", key[2])]))
    rets = self.service_table.entry_get(self.target,
                                                   key_list,
                                                   flags={"from_hw": True})
    for ret in rets:
        data = ret[0].to_dict()
        key = ret[1].to_dict()
        if "$ENTRY_TTL" == 0:
            expired_entries.append([key['meta.internal_ip']['value'],
                                    key['meta.internal_port']['value'],
                                    key['meta.ip_protocol']['value']])
    return expired_entries

start_time = time.time()
expired_entries = service_entry_get(controller, test_key_list[:2000])

print("get --- %s seconds ---" % (time.time() - start_time))

get --- 0.5595502853393555 seconds ---


In [6]:
# single clean time
start_time = time.time()
controller.idle_entry_single_clean()
print("single clean --- %s seconds ---" % (time.time() - start_time))
print(controller.get_table_usage())

Idletime notification not received
Removed 300 idle entries, cost 0.83s!
single clean --- 0.8259332180023193 seconds ---
0


In [21]:
random.shuffle(test_key_list)

In [22]:
print("add entries again, length: ", len(test_key_list))
controller.entry_batch_add(test_key_list)
print(controller.get_table_usage())

add entries again, length:  300
0
